In [4]:
import streamlit as st
from PIL import Image
import cv2 
import numpy as np 
import random 

#---------------------------------------------------------
# Text Encoding/Decoding
#---------------------------------------------------------
def genData(data):
    newd = []
    for i in data:
        newd.append(format(ord(i), '08b'))
    return newd

def modPix(pix, data):
    datalist = genData(data)
    lendata = len(datalist)
    imdata = iter(pix)

    for i in range(lendata):
        pix = [value for value in imdata.__next__()[:3] +
                                imdata.__next__()[:3] +
                                imdata.__next__()[:3]]

        for j in range(0, 8):
            if (datalist[i][j] == '0' and pix[j] % 2 != 0):
                pix[j] -= 1
            elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
                if(pix[j] != 0):
                    pix[j] -= 1
                else:
                    pix[j] += 1

        if (i == lendata - 1):
            if (pix[-1] % 2 == 0):
                if(pix[-1] != 0):
                    pix[-1] -= 1
                else:
                    pix[-1] += 1
        else:
            if (pix[-1] % 2 != 0):
                pix[-1] -= 1

        pix = tuple(pix)
        yield pix[0:3]
        yield pix[3:6]
        yield pix[6:9]

def encode_enc(newimg, data):
    w = newimg.size[0]
    (x, y) = (0, 0)

    for pixel in modPix(newimg.getdata(), data):
        newimg.putpixel((x, y), pixel)
        if (x == w - 1):
            x = 0
            y += 1
        else:
            x += 1

def encode(image, data, new_img_name):
    if (len(data) == 0):
        raise ValueError('Data is empty')

    newimg = image.copy()
    encode_enc(newimg, data)
    
    print("new_img_name:")
    print(new_img_name)
    
    newimg.save(new_img_name, str(new_img_name.split(".")[1].upper()))

def decode(image):
    data = ''
    imgdata = iter(image.getdata())

    while (True):
        pixels = [value for value in imgdata.__next__()[:3] +
                                imgdata.__next__()[:3] +
                                imgdata.__next__()[:3]]

        binstr = ''

        for i in pixels[:8]:
            if (i % 2 == 0):
                binstr += '0'
            else:
                binstr += '1'

        data += chr(int(binstr, 2))
        if (pixels[-1] % 2 != 0):
            return data
          
if 'clicked' not in st.session_state:
    st.session_state.clicked = False
    
def click_button():
    st.session_state.clicked = True

#---------------------------------------------------------
# Image Encoding/Decoding
#---------------------------------------------------------
def image_encryption(c_image_file, h_image_file, e_image_name): 
      
    # img1 and img2 are the 
    # two input images 
    img1 = cv2.imread(str(c_image_file))
    img2 = cv2.imread(str(h_image_file))
      
    for i in range(img2.shape[0]): 
        for j in range(img2.shape[1]): 
            for l in range(3): 
                  
                # v1 and v2 are 8-bit pixel values 
                # of img1 and img2 respectively 
                v1 = format(img1[i][j][l], '08b') 
                v2 = format(img2[i][j][l], '08b') 
                  
                # Taking 4 MSBs of each image 
                v3 = v1[:4] + v2[:4]  
                  
                img1[i][j][l]= int(v3, 2) 
                  
    cv2.imwrite(e_image_name, img1)
    
def image_decryption(d_image_file, d_img1, d_img2): 
      
    # Encrypted image 
    img = cv2.imread(str(d_image_file))  
    width = img.shape[0] 
    height = img.shape[1] 
      
    # img1 and img2 are two blank images 
    img1 = np.zeros((width, height, 3), np.uint8) 
    img2 = np.zeros((width, height, 3), np.uint8) 
      
    for i in range(width): 
        for j in range(height): 
            for l in range(3): 
                v1 = format(img[i][j][l], '08b') 
                v2 = v1[:4] + chr(random.randint(0, 1)+48) * 4
                v3 = v1[4:] + chr(random.randint(0, 1)+48) * 4
                  
                # Appending data to img1 and img2 
                img1[i][j][l]= int(v2, 2) 
                img2[i][j][l]= int(v3, 2) 
      
    # These are two images produced from 
    # the encrypted image 
    cv2.imwrite(d_img1, img1) 
    cv2.imwrite(d_img2, img2)
    

#---------------------------------------------------------
# Streamlit UI
#---------------------------------------------------------
def main():
    
    st.title("Steganography 🔒")
    text_encoding_tab, image_encryption_tab = st.tabs(["Text  🖋️", "Image  [ ◉¯]"])
    
    # Text UI
    with text_encoding_tab:
        st.subheader("Text Encoder/Decoder")
        option = st.selectbox("Select Option", ("Encode Text", "Decode Text"))

        if option == "Encode Text":
            st.markdown("#### Encode Message in Image")
            img_file = st.file_uploader("Upload Image", type=["jpg", "png", "jpeg"])

            if img_file is not None:
                image = Image.open(img_file, 'r')
                data = st.text_input("Enter Message to be Encoded")
                new_img_name = st.text_input("Enter Encoded Image Name (with extension)")
                st.button("Submit", on_click=click_button)

                if st.session_state.clicked:
                    print("submit")
                    encode(image, data, new_img_name)
                    st.success("Message Encoded Successfully!")

        elif option == "Decode Text":
            st.markdown("#### Decode Message from Image")
            img_file = st.file_uploader("Upload Encoded Image", type=["jpg", "png"])

            if img_file is not None:
                image = Image.open(img_file, 'r')

                if st.button("Decode"):
                    decoded_data = decode(image)
                    st.success(f"Decoded Message: {decoded_data}")
    # Image UI                
    with image_encryption_tab:
        st.subheader("Image Encryption/Decryption")
        option = st.selectbox("Select Option", ("Encrypt Image", "Decrypt Image"))
        
        if option == "Encrypt Image":
            st.markdown("#### Encrypt Image")
            cover_image_file = st.file_uploader("Upload Cover Image", type=["jpg", "png", "jpeg", "tiff"])
            hidden_image_file = st.file_uploader("Upload Hidden Image", type=["jpg", "png", "jpeg", "tiff"])

            if cover_image_file and hidden_image_file is not None:
                encrypted_image_name = st.text_input("Enter Encrypted Image Name (with extension)")
                st.button("Encrypt", on_click=click_button)

                if st.session_state.clicked:
                    image_encryption(cover_image_file.name, hidden_image_file.name, encrypted_image_name)
                    st.success("Image Encrypted Successfully!")
                    
        elif option == "Decrypt Image":
            st.markdown("#### Decrypt Image")
            decrypt_image_file = st.file_uploader("Upload Encrypted Image", type=["jpg", "png", "jpeg", "tiff"])
            
            if decrypt_image_file is not None:
                decrypted_image_1_name = st.text_input("Enter First Decrypted Image Name (with extension)")
                decrypted_image_2_name = st.text_input("Enter Second Decrypted Image Name (with extension)")
                st.button("Decrypt", on_click=click_button)
                
                if st.session_state.clicked:
                    image_decryption(decrypt_image_file.name, decrypted_image_1_name, decrypted_image_2_name)
                    st.success("Image Decrpyted Succesfully!")
                
if __name__ == "__main__":
    main()


In [9]:
!jupytext --to .py Steganography.ipynb

[jupytext] Reading Steganography.ipynb in format ipynb
[jupytext] Writing Steganography.py (destination file replaced)


In [10]:
!streamlit run Steganography.py --server.enableXsrfProtection false


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.21:8501

^C
  Stopping...
